<a href="https://colab.research.google.com/github/epalosh/SEES-2022/blob/main/Ground_Surface_Temperature_And_Elevation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Land Surface Temperature Using GEE!

*Notebook by: Ethan Palosh*

Dataset: "PRISM Daily Spatial Climate Dataset AN81d" 
https://developers.google.com/earth-engine/datasets/catalog/OREGONSTATE_PRISM_AN81d#bands 

Topics practiced:
  - Reducers
  - Loading and filtering image collections
  - Loading, filtering, and editing vector data

## Setup

### Earth Engine setup

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=v-M3nDaPwpWbQBZSPjgy3-HG3uKBXuboH33npwAVBFk&tc=AaylCc0H1BVXXK46SIlj_5kQKRctdjlSBMJTHoVomeI&cc=wl_PGvWIkcSM9PN-ix_sp-HF83NapGdaPgFnYzb9UmA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qg5Np-VxLlH7j-H_8USPlL2QU2CETHAY1V3l8l-MUQ2IiDCIT5rUOY

Successfully saved authorization token.


### GEE map setup (for interactive map display)

In [2]:
# Installs geemap package
import subprocess

try:
    import geemap.eefolium as geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap.eefolium as geemap
    print("geemap is now installed, imported and ready to use in Colab") 

#geemap.update_package()

geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [3]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Code!

**Create Image collections!**

First, I imported data, selected a band, and filtered by date.

Then, I created another image collection of ecoregions.

Finally, I reduced the dataset by "mean" to find the average of daily averages over the time period!

In [4]:
# load temp data through PRISM: 365 images per year
precipCollection = ee.ImageCollection('OREGONSTATE/PRISM/AN81d') \
                    .select('tmean') \
                    .filterDate('2021-01-01', '2021-12-31')

EcoRegions = ee.FeatureCollection('EPA/Ecoregions/2013/L3') 
   # .filter(ee.Filter.eq("STATEFP", "48"))


avgTemp = precipCollection.reduce(ee.Reducer.mean())

**Filter and define by areas of interest...**

In this cell I filter by the states I'm looking to display county averages. I use the states that my group members are from: Washington, California, Georgia, Texas, and New Jersey.

Then, I reduce the avgTemp image to display average temp within a county.

Finally, I created an image that can be layered ontop of other maps that displays the ecoregions I pulled earlier.

In [5]:
SEES_CD_States = ['06','13','34','48','53'] # state FIPS codes that we want

# a new Feature Collection of only the states that we want to analyze
counties = ee.FeatureCollection("TIGER/2018/Counties") \
        .filter(ee.Filter.inList('STATEFP', SEES_CD_States))

# get mean temp values by county polygon
# Note the use of the reduceRegions and the mean Reducer.
# 4638.3 is the resolution of the PRISM dataset

countyTemp = avgTemp.reduceRegions(
    **{
        'collection': counties,
        'reducer': ee.Reducer.mean(),
        'scale': 4638.3,
       }
    )

#paint ecoregions onto map!
ECO = ee.Image().paint(EcoRegions, 0, 1)

In [6]:
tiger_co = ee.FeatureCollection("TIGER/2018/Counties")
print(type(tiger_co))

<class 'ee.featurecollection.FeatureCollection'>


Calculates the highest county mean temperature and show value. (For use in defining bounds when mapping)

In [7]:
our_max_mean = countyTemp.aggregate_max('mean')
print('Max of mean county average temperature over a year = {} degrees celsius'.format(our_max_mean.getInfo()))

Max of mean county average temperature over a year = 24.157193714831102 degrees celsius


Calculates the lowest county mean temperature and show value.

In [8]:
our_min_mean = countyTemp.aggregate_min('mean')
print('Min of mean county average temperature over a year = {} degrees celsius'.format(our_min_mean.getInfo()))

Min of mean county average temperature over a year = 6.547634910029122 degrees celsius


##Show the resulting mean average temperature by county

In this code cell, I start by defning a color palette for the map.

Then, I defined a painting variable called "fills" which will map the county temperatures by averaging the values in the county, and mapping it withing the boundries. (like little blocks of color)

In [10]:
palette = ['purple','blue','cyan','green','yellow','red']
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()
fills = empty.paint(
    **{
        'featureCollection': countyTemp,
        'color': 'mean',
       }
    )

**Finally, I map it!**

First, I define the map variable and set the center and zoom factor.

Then, I add each layer to the map. Access each layer by selecting the layer button on the top right of the map.

In [13]:
Map = geemap.Map()
Map.setCenter(-99.67, 38.41, 5)
# Note using min and max from aggregate functions to set high and low visualization values.
Map.addLayer(fills, {'palette': palette, 'min': 2, 'max': 24}, 'Country Averages')
Map.addLayer(avgTemp, {'palette': palette, 'min': 2, 'max': 24}, 'national colored fills')
Map.addLayer(ECO, {}, 'Eco Regions') #Add ecoregion boundries

Map

# Elevation data!

For fun, I decided to map elevation data as well, on a greyscale. The darker areas represent higher elevations.

This map is unique because it utilizes a "color ramp" with defined values. If needed, I can define additional values (and associated colors) to represent data more precisely in certain ranges while preserving the range.

I could define colors at 100, 200, 300, 400, and 3000.

In this example though, I just linearly mapped a greyscale.

In [23]:
# Load SRTM Digital Elevation Model data.
image = ee.Image('CGIAR/SRTM90_V4')

# Define an SLD style of discrete intervals to apply to the image.
sld_ramp = """
    <RasterSymbolizer>
      <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#ffffff" quantity="100" label="0"/>
        <ColorMapEntry color="#000000" quantity="3500" label="1-3500" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define a map centered on the United States.
map_sld_interval = folium.Map(location=[40.413, -99.229], zoom_start=5)

# Add the image layers to the map and display it.
# map_sld_interval.add_ee_layer(
#     image.sldStyle(sld_intervals), None, 'SLD intervals')
map_sld_interval.add_ee_layer(image.sldStyle(sld_ramp), None, 'SLD ramp')
display(map_sld_interval.add_child(folium.LayerControl()))

##Vector to Raster Conversion

*Not written by me. Used to convert map into different format!* 

*Credit: Teresa Howard*


Why convert the county polygon vectors to a raster? <br>
**Vocabulary to remember:**

*   vector = point, line, polygon = *ee* Feature
*   raster = gridded data = *ee* Image
<br>
<br>
*ee* offers more visualization options for Images than for Features





In [25]:
#Make an image out of the mean temperature attribute.
meanTempImg = countyTemp.reduceToImage(properties=['mean'], reducer=ee.Reducer.first())

# Make a stylized palette color map
Map = geemap.Map()
Map.setCenter(-99.67, 38.41, 5)
Map.addLayer(meanTempImg, {'palette': palette, 'max': 24}, 'County means')
Map.addLayer(avgTemp, {'min': 1, 'max': 24, 'palette': palette}, 'PRISM Temp')

Map